In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/santandar-processed-data/fdata.csv


In [2]:
data = pd.read_csv("/kaggle/input/santandar-processed-data/fdata.csv")

In [3]:
data.head()

,data_date,employee_index,country_residence,gender,registration_date,new_customer_index,seniority,primary_customer,customer_type_beginning_month,customer_relation_type_beginning_month,...,prod_pensions1,prod_loans,prod_taxes,prod_credit_card,prod_securities,prod_home_account,prod_payroll,prod_pensions2,prod_direct_debit,age_group
0,2015-01-28,3,0,0,2015-01-12,0,6.0,1.0,1,A,...,0,0,0,0,0,0,0.0,0.0,0,1
1,2015-01-28,3,0,1,2012-08-10,0,35.0,1.0,1,I,...,0,0,0,0,0,0,0.0,0.0,0,1
2,2015-01-28,3,0,1,2012-08-10,0,35.0,1.0,1,I,...,0,0,0,0,0,0,0.0,0.0,0,1
3,2015-01-28,3,0,0,2012-08-10,0,35.0,1.0,1,I,...,0,0,0,0,0,0,0.0,0.0,0,1
4,2015-01-28,3,0,1,2012-08-10,0,35.0,1.0,1,A,...,0,0,0,0,0,0,0.0,0.0,0,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13618663 entries, 0 to 13618662
Data columns (total 43 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   data_date                               object 
 1   employee_index                          int64  
 2   country_residence                       int64  
 3   gender                                  int64  
 4   registration_date                       object 
 5   new_customer_index                      int64  
 6   seniority                               float64
 7   primary_customer                        float64
 8   customer_type_beginning_month           int64  
 9   customer_relation_type_beginning_month  object 
 10  residence_index                         object 
 11  foreigner_index                         object 
 12  channel_used                            object 
 13  province_code                           float64
 14  province_name                   

In [5]:
# Import ML libraries
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [6]:
product_list = [
    'prod_savings_account',
    'prod_guarantees',
    'prod_current_accounts',
    'prod_derivative_account',
    'prod_payroll_account',
    'prod_junior_account',
    'prod_mas_particular_account',
    'prod_particular_account',
    'prod_particular_plus_account',
    'prod_short_term_deposits',
    'prod_medium_term_deposits',
    'prod_long_term_deposits',
    'prod_e_account',
    'prod_funds',
    'prod_mortgage',
    'prod_pensions1',
    'prod_loans',
    'prod_taxes',
    'prod_credit_card',
    'prod_securities',
    'prod_home_account',
    'prod_payroll',
    'prod_pensions2',
    'prod_direct_debit'
]


In [7]:
# Convert date columns to datetime format
from sklearn.preprocessing import LabelEncoder
date_columns = ['data_date', 'registration_date']
for col in date_columns:
    print(col)
    data[col] = pd.to_datetime(data[col])

# Encode categorical variables
categorical_columns = ['customer_relation_type_beginning_month', 'residence_index', 'foreigner_index', 'channel_used', 'province_name', 'segmentation']
label_encoders = {}
for col in categorical_columns:
    print(col)
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col].astype(str))

data_date
registration_date
customer_relation_type_beginning_month
residence_index
foreigner_index
channel_used
province_name
segmentation


In [8]:
for col in date_columns:
    print(col)
    data[col] = (data[col] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

data_date
registration_date


In [9]:
data.isnull().sum()

data_date                                   0
employee_index                              0
country_residence                           0
gender                                      0
registration_date                           0
new_customer_index                          0
seniority                                   0
primary_customer                            0
customer_type_beginning_month               0
customer_relation_type_beginning_month      0
residence_index                             0
foreigner_index                             0
channel_used                                0
province_code                               0
province_name                               0
activity_index                              0
gross_income_household                      0
segmentation                                0
prod_savings_account                        0
prod_guarantees                             0
prod_current_accounts                       0
prod_derivative_account           

In [10]:
import pandas as pd

# Assuming 'df' is your DataFrame

# Extract rows where gross_income value is 2000
condition = (data['gross_income_household'] == 101850) & (data['data_date']==1422403200) & (data['registration_date']==1295913600) & (data['channel_used']==24) & (data['province_code']==28)
rows_with_2000_income = data[condition]
new_df = pd.DataFrame(rows_with_2000_income)
# Display the extracted rows
new = new_df.loc[:,'prod_savings_account':]
new


,prod_savings_account,prod_guarantees,prod_current_accounts,prod_derivative_account,prod_payroll_account,prod_junior_account,prod_mas_particular_account,prod_particular_account,prod_particular_plus_account,prod_short_term_deposits,...,prod_pensions1,prod_loans,prod_taxes,prod_credit_card,prod_securities,prod_home_account,prod_payroll,prod_pensions2,prod_direct_debit,age_group
99517,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0,2
99541,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0,2


In [11]:
import pandas as pd

# Assuming 'df' is your DataFrame

# Iterate through DataFrame rows and print values
for index, row in new.iterrows():
    print(f"Row index: {index}")
    for column, value in row.items():
        print(f"Column: {column}, Value: {value}")


Row index: 99517
Column: prod_savings_account, Value: 0.0
Column: prod_guarantees, Value: 0.0
Column: prod_current_accounts, Value: 1.0
Column: prod_derivative_account, Value: 0.0
Column: prod_payroll_account, Value: 0.0
Column: prod_junior_account, Value: 0.0
Column: prod_mas_particular_account, Value: 0.0
Column: prod_particular_account, Value: 0.0
Column: prod_particular_plus_account, Value: 0.0
Column: prod_short_term_deposits, Value: 0.0
Column: prod_medium_term_deposits, Value: 0.0
Column: prod_long_term_deposits, Value: 1.0
Column: prod_e_account, Value: 0.0
Column: prod_funds, Value: 0.0
Column: prod_mortgage, Value: 0.0
Column: prod_pensions1, Value: 0.0
Column: prod_loans, Value: 0.0
Column: prod_taxes, Value: 0.0
Column: prod_credit_card, Value: 0.0
Column: prod_securities, Value: 0.0
Column: prod_home_account, Value: 0.0
Column: prod_payroll, Value: 0.0
Column: prod_pensions2, Value: 0.0
Column: prod_direct_debit, Value: 0.0
Column: age_group, Value: 2.0
Row index: 99541
Co

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13618663 entries, 0 to 13618662
Data columns (total 43 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   data_date                               int64  
 1   employee_index                          int64  
 2   country_residence                       int64  
 3   gender                                  int64  
 4   registration_date                       int64  
 5   new_customer_index                      int64  
 6   seniority                               float64
 7   primary_customer                        float64
 8   customer_type_beginning_month           int64  
 9   customer_relation_type_beginning_month  int64  
 10  residence_index                         int64  
 11  foreigner_index                         int64  
 12  channel_used                            int64  
 13  province_code                           float64
 14  province_name                   

In [13]:
data_without_missing = data.dropna()

In [14]:
data_without_missing.isnull().sum()

data_date                                 0
employee_index                            0
country_residence                         0
gender                                    0
registration_date                         0
new_customer_index                        0
seniority                                 0
primary_customer                          0
customer_type_beginning_month             0
customer_relation_type_beginning_month    0
residence_index                           0
foreigner_index                           0
channel_used                              0
province_code                             0
province_name                             0
activity_index                            0
gross_income_household                    0
segmentation                              0
prod_savings_account                      0
prod_guarantees                           0
prod_current_accounts                     0
prod_derivative_account                   0
prod_payroll_account            

In [15]:
data_without_missing.head()

,data_date,employee_index,country_residence,gender,registration_date,new_customer_index,seniority,primary_customer,customer_type_beginning_month,customer_relation_type_beginning_month,...,prod_pensions1,prod_loans,prod_taxes,prod_credit_card,prod_securities,prod_home_account,prod_payroll,prod_pensions2,prod_direct_debit,age_group
0,1422403200,3,0,0,1421020800,0,6.0,1.0,1,0,...,0,0,0,0,0,0,0.0,0.0,0,1
1,1422403200,3,0,1,1344556800,0,35.0,1.0,1,1,...,0,0,0,0,0,0,0.0,0.0,0,1
2,1422403200,3,0,1,1344556800,0,35.0,1.0,1,1,...,0,0,0,0,0,0,0.0,0.0,0,1
3,1422403200,3,0,0,1344556800,0,35.0,1.0,1,1,...,0,0,0,0,0,0,0.0,0.0,0,1
4,1422403200,3,0,1,1344556800,0,35.0,1.0,1,0,...,0,0,0,0,0,0,0.0,0.0,0,1


In [16]:
data_without_missing.shape

(13618446, 43)

In [17]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import xgboost as xgb


X = data_without_missing.loc[:, 'data_date':'segmentation'][:100000]
y = data_without_missing.loc[:, 'prod_savings_account':][:100000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
pred = {}

params = {
    'objective': 'binary:logistic',  
    'eval_metric': 'logloss',         
    'max_depth': 3,                   
    'learning_rate': 0.1,             
    'n_estimators': 300,              
    'random_state': 123               
}

for col in tqdm(product_list):
    
    if data[col].nunique() == 2:
        y = y_test[col]
        xgb_model = xgb.XGBClassifier(**params)
        xgb_model.fit(X_train, y_train[col])
        xgb_probabilities = xgb_model.predict_proba(X_test)[:, 1]
        pred[col] = xgb_probabilities
    else:
        pass


100%|██████████| 24/24 [00:34<00:00,  1.42s/it]


In [18]:
X_train
X_test

,data_date,employee_index,country_residence,gender,registration_date,new_customer_index,seniority,primary_customer,customer_type_beginning_month,customer_relation_type_beginning_month,residence_index,foreigner_index,channel_used,province_code,province_name,activity_index,gross_income_household,segmentation
42084,1422403200,3,0,1,1354838400,0,31.0,1.0,1,1,1,0,150,8.0,8,0.0,101850.00,0
71829,1422403200,3,0,0,1351814400,0,32.0,1.0,1,1,1,0,150,3.0,2,0.0,82711.11,0
99541,1422403200,3,0,0,1295913600,0,54.0,1.0,1,0,1,0,24,28.0,30,1.0,101850.00,1
47881,1422403200,3,0,1,1352764800,0,32.0,1.0,1,1,1,0,150,28.0,30,0.0,95567.43,0
36735,1422403200,3,0,1,1343347200,0,36.0,1.0,1,0,1,0,150,45.0,47,1.0,131097.15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93923,1422403200,3,0,1,1315785600,0,46.0,1.0,1,1,1,0,150,8.0,8,0.0,101850.00,0
42001,1422403200,3,0,1,1354665600,0,31.0,1.0,1,0,1,1,122,17.0,21,1.0,43860.90,1
35032,1422403200,3,0,1,1343174400,0,36.0,1.0,1,1,1,0,150,36.0,38,0.0,101850.00,0
42196,1422403200,3,0,0,1354665600,0,31.0,1.0,1,1,1,1,150,33.0,4,0.0,61956.09,0


In [19]:
pred

{'prod_savings_account': array([1.2019697e-05, 1.2019697e-05, 1.2019697e-05, ..., 1.2019697e-05,
        1.2019697e-05, 1.2019697e-05], dtype=float32),
 'prod_guarantees': array([1.2019697e-05, 1.2019697e-05, 1.2019697e-05, ..., 1.2019697e-05,
        1.2019697e-05, 1.2019697e-05], dtype=float32),
 'prod_current_accounts': array([0.99525154, 0.994487  , 0.6076131 , ..., 0.9961092 , 0.9979048 ,
        0.7061314 ], dtype=float32),
 'prod_derivative_account': array([1.1950946e-05, 7.6739179e-06, 1.7769229e-05, ..., 1.1292352e-05,
        8.0752870e-06, 5.3400607e-05], dtype=float32),
 'prod_payroll_account': array([0.00165112, 0.00268743, 0.2249965 , ..., 0.00065126, 0.00095401,
        0.2505436 ], dtype=float32),
 'prod_junior_account': array([1.3129008e-06, 7.1604641e-06, 9.4671816e-02, ..., 4.2878160e-06,
        1.3301720e-06, 2.0147691e-02], dtype=float32),
 'prod_mas_particular_account': array([9.521378e-05, 4.585971e-05, 4.841051e-01, ..., 5.665202e-05,
        7.248892e-05, 2.06

In [20]:
pred_df = pd.DataFrame(pred)

data_subset = X_test

concatenated_data = pd.concat([data_subset.reset_index(drop=True), pred_df], axis=1)

In [21]:
X_test.shape

(20000, 18)

In [22]:
data_subset.head()

,data_date,employee_index,country_residence,gender,registration_date,new_customer_index,seniority,primary_customer,customer_type_beginning_month,customer_relation_type_beginning_month,residence_index,foreigner_index,channel_used,province_code,province_name,activity_index,gross_income_household,segmentation
42084,1422403200,3,0,1,1354838400,0,31.0,1.0,1,1,1,0,150,8.0,8,0.0,101850.00,0
71829,1422403200,3,0,0,1351814400,0,32.0,1.0,1,1,1,0,150,3.0,2,0.0,82711.11,0
99541,1422403200,3,0,0,1295913600,0,54.0,1.0,1,0,1,0,24,28.0,30,1.0,101850.00,1
47881,1422403200,3,0,1,1352764800,0,32.0,1.0,1,1,1,0,150,28.0,30,0.0,95567.43,0
36735,1422403200,3,0,1,1343347200,0,36.0,1.0,1,0,1,0,150,45.0,47,1.0,131097.15,0


In [23]:
pred_df.head()

,prod_savings_account,prod_guarantees,prod_current_accounts,prod_derivative_account,prod_payroll_account,prod_junior_account,prod_mas_particular_account,prod_particular_account,prod_particular_plus_account,prod_short_term_deposits,...,prod_mortgage,prod_pensions1,prod_loans,prod_taxes,prod_credit_card,prod_securities,prod_home_account,prod_payroll,prod_pensions2,prod_direct_debit
0,0.000012,0.000012,0.995252,0.000012,0.001651,0.000001,0.000095,0.000002,1.829232e-06,6.550881e-06,...,1.913893e-06,0.000027,1.989192e-05,0.001693,0.000110,0.000012,0.000005,0.000200,0.000236,0.001600
1,0.000012,0.000012,0.994487,0.000008,0.002687,0.000007,0.000046,0.000004,3.844946e-07,1.727203e-06,...,7.213220e-07,0.000002,4.642431e-06,0.001096,0.000054,0.000022,0.000003,0.000137,0.000195,0.003040
2,0.000012,0.000012,0.607613,0.000018,0.224997,0.094672,0.484105,0.001166,3.313671e-04,3.761074e-04,...,1.015433e-05,0.009680,1.733187e-05,0.108847,0.183779,0.015455,0.000048,0.181655,0.187033,0.355620
3,0.000012,0.000012,0.994366,0.000014,0.002002,0.000004,0.000086,0.000013,3.645002e-07,7.964283e-07,...,8.254537e-07,0.000026,9.449055e-07,0.001364,0.000212,0.000021,0.000005,0.000310,0.000243,0.002697
4,0.000012,0.000012,0.941517,0.000128,0.052029,0.000006,0.000097,0.000004,7.580739e-05,1.575079e-05,...,2.895454e-05,0.000127,7.017625e-07,0.005753,0.002459,0.002318,0.000013,0.012405,0.012691,0.097756


In [24]:
def gbdt_product_recommender(df, row_number, top_n):
    
    prod_list = df.iloc[row_number, -len(pred_df.columns):].reset_index()
    prod_list.columns = ['product', 'pred_score']
    prod_list = prod_list.sort_values(by='pred_score', ascending=False)
    
    prod_list = prod_list[prod_list['pred_score'] >= 0.000001]
    recommend_list = prod_list.head(top_n)
    
    if recommend_list.empty:
        print("Based on the customer's info, there is no bank product recommended for now")
    
    return recommend_list


In [25]:
gbdt_product_recommender(concatenated_data ,11,10)

,product,pred_score
2,prod_current_accounts,0.687846
6,prod_mas_particular_account,0.389911
23,prod_direct_debit,0.310959
12,prod_e_account,0.183600
4,prod_payroll_account,0.156155
18,prod_credit_card,0.117317
21,prod_payroll,0.112643
22,prod_pensions2,0.111711
11,prod_long_term_deposits,0.101295
17,prod_taxes,0.025590


In [26]:
concatenated_data.head()

,data_date,employee_index,country_residence,gender,registration_date,new_customer_index,seniority,primary_customer,customer_type_beginning_month,customer_relation_type_beginning_month,...,prod_mortgage,prod_pensions1,prod_loans,prod_taxes,prod_credit_card,prod_securities,prod_home_account,prod_payroll,prod_pensions2,prod_direct_debit
0,1422403200,3,0,1,1354838400,0,31.0,1.0,1,1,...,1.913893e-06,0.000027,1.989192e-05,0.001693,0.000110,0.000012,0.000005,0.000200,0.000236,0.001600
1,1422403200,3,0,0,1351814400,0,32.0,1.0,1,1,...,7.213220e-07,0.000002,4.642431e-06,0.001096,0.000054,0.000022,0.000003,0.000137,0.000195,0.003040
2,1422403200,3,0,0,1295913600,0,54.0,1.0,1,0,...,1.015433e-05,0.009680,1.733187e-05,0.108847,0.183779,0.015455,0.000048,0.181655,0.187033,0.355620
3,1422403200,3,0,1,1352764800,0,32.0,1.0,1,1,...,8.254537e-07,0.000026,9.449055e-07,0.001364,0.000212,0.000021,0.000005,0.000310,0.000243,0.002697
4,1422403200,3,0,1,1343347200,0,36.0,1.0,1,0,...,2.895454e-05,0.000127,7.017625e-07,0.005753,0.002459,0.002318,0.000013,0.012405,0.012691,0.097756


In [27]:
tenth_row = concatenated_data.iloc[100]

In [28]:
tenth_row_as_df = concatenated_data.iloc[[100]]
tenth_row_as_df = pd.DataFrame(tenth_row_as_df).T

In [29]:
pd.set_option('display.max_rows', None) 
print(tenth_row_as_df)

                                                 100
data_date                               1.422403e+09
employee_index                          3.000000e+00
country_residence                       0.000000e+00
gender                                  1.000000e+00
registration_date                       1.354234e+09
new_customer_index                      0.000000e+00
seniority                               3.100000e+01
primary_customer                        1.000000e+00
customer_type_beginning_month           1.000000e+00
customer_relation_type_beginning_month  1.000000e+00
residence_index                         1.000000e+00
foreigner_index                         0.000000e+00
channel_used                            1.500000e+02
province_code                           2.800000e+01
province_name                           3.000000e+01
activity_index                          0.000000e+00
gross_income_household                  8.799324e+04
segmentation                            0.0000

In [30]:
concatenated_data.isnull().sum()

data_date                                 0
employee_index                            0
country_residence                         0
gender                                    0
registration_date                         0
new_customer_index                        0
seniority                                 0
primary_customer                          0
customer_type_beginning_month             0
customer_relation_type_beginning_month    0
residence_index                           0
foreigner_index                           0
channel_used                              0
province_code                             0
province_name                             0
activity_index                            0
gross_income_household                    0
segmentation                              0
prod_savings_account                      0
prod_guarantees                           0
prod_current_accounts                     0
prod_derivative_account                   0
prod_payroll_account            

In [31]:
pred_prob = pred_df.iloc[2:3,:]
pl = pred_prob.values.tolist()
pl = pl[0]

# Thompson Sampling



In [32]:
import numpy as np

# Thompson Sampling function
def thompson_sampling(prod_prob, num_samples):
    rewards = np.zeros(len(prod_prob))  # Initialize rewards for each product
    selections = np.zeros(len(prod_prob))  # Initialize counts of selections for each product
    
    # Iterate for num_samples
    for _ in range(num_samples):
        # Sample from beta distribution for each product
        sampled_values = [np.random.beta(rewards[i] + 1, selections[i] - rewards[i] + 1) for i in range(len(prod_prob))]
        
        # Select the product with the highest sampled value
        selected_product = np.argmax(sampled_values)
        
        # Update rewards and selections
        reward = np.random.choice([0, 1], p=[1 - prod_prob[selected_product], prod_prob[selected_product]])
        rewards[selected_product] += reward
        selections[selected_product] += 1
    
    return rewards, selections

# Example usage
prod_prob = pl  # Example probabilities for products
num_samples = 10000  # Number of iterations for Thompson Sampling

# Run Thompson Sampling
rewards, selections = thompson_sampling(prod_prob, num_samples)

# Print results
print("Estimated rewards for each product:", rewards)
print("Number of selections for each product:", selections)


Estimated rewards for each product: [0.000e+00 0.000e+00 5.855e+03 0.000e+00 1.000e+00 0.000e+00 4.200e+01
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 1.000e+00 0.000e+00 0.000e+00 7.000e+00 1.000e+00 0.000e+00
 5.000e+00 4.000e+00 1.500e+01]
Number of selections for each product: [8.000e+00 8.000e+00 9.628e+03 9.000e+00 1.200e+01 9.000e+00 9.300e+01
 9.000e+00 9.000e+00 9.000e+00 8.000e+00 9.000e+00 8.000e+00 8.000e+00
 9.000e+00 1.200e+01 1.000e+01 1.000e+01 2.800e+01 1.200e+01 8.000e+00
 2.300e+01 1.900e+01 4.200e+01]


In [33]:
for i in pl:
    print(i)

1.201969735120656e-05
1.201969735120656e-05
0.6076130867004395
1.776922908902634e-05
0.22499650716781616
0.0946718156337738
0.48410511016845703
0.0011659030569717288
0.00033136707497760653
0.00037610740400850773
0.0017202498856931925
0.0781480073928833
0.11998790502548218
0.015886005014181137
1.0154325536859687e-05
0.009679737500846386
1.733187491481658e-05
0.10884738713502884
0.1837785691022873
0.015454590320587158
4.847352101933211e-05
0.18165472149848938
0.1870332509279251
0.3556196093559265
